<div class="alert alert-info"> 💡 <strong> Note </strong>
This notebook was tested with the TensorFlow 1.15 Python 3.7 (optimized for CPU) image and Python 3 kernel. Please make sure the correct kernel and image is et before executing this notebook
</div>

# Notebook 1 - Financial News Sentiment Analysis

## Use case
Sentiment Analysis of Financial news provides practical guidance on the use of sentiment data in investment and trading strategies. It can assist financial trading and automatic trading systems to make investment decisions and enhance their predictive performance. Understanding and measuring the relationship of financial news with sentiment data can help analyst come up with financial forecasting. 

Financial news sentiment analysis is a type of natural language processing (NLP) that uses machine learning algorithms to analyze and classify the sentiment of financial news articles or other related content. The goal is to determine the overall sentiment of the news and identify whether the sentiment is positive, negative, or neutral.

This analysis can help investors and traders make more informed decisions by identifying patterns in the sentiment of news articles that may influence financial markets. For example, if there is a lot of negative sentiment surrounding a particular company or industry, it could be an indication that the market will react negatively.

Sentiment analysis in finance can also be used to track the performance of companies or financial products over time. By analyzing news articles and other sources of financial information, investors and traders can gain insight into how the market perceives certain companies or products.

Overall, financial news sentiment analysis is a valuable tool for investors and traders who want to stay on top of market trends and make informed decisions based on the latest news and sentiment data.

## What will you learn?

In this notebook, we will walk you through how to use NVIDIA Triton Inference Server on Amazon SageMaker multi-model endpoints(MME) with GPU feature to deploy NLP models (**DistilBERT**) for financial news sentiment analysis.

Amazon SageMaker MME provide a scalable and cost-effective way to deploy large number of deep learning models. Previously, customers had limited options to deploy 100s of deep learning models that need accelerated compute with GPUs. Now customers can deploy 1000s of deep learning models behind one SageMaker endpoint. Now, MME will run multiple models on a GPU, share GPU instances behind an endpoint across multiple models and dynamically load/unload models based on the incoming traffic. With this, customers can significantly save cost and achieve best price performance.

## Installs

Installs the dependencies required to package the model and run inferences using Triton server. Update SageMaker, boto3, awscli etc

In [ ]:
!pip install -qU pip awscli boto3 sagemaker
!pip install nvidia-pyindex --quiet
!pip install tritonclient[http] --quiet
!pip install transformers[sentencepiece] --quiet
!pip install tensorflow

## Imports and variables

In [ ]:
import boto3, json, sagemaker, time
from sagemaker import get_execution_role
import numpy as np
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# sagemaker variables
role = get_execution_role()
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client("sagemaker-runtime")
sagemaker_session = sagemaker.Session(boto_session=boto3.Session())
s3_client = boto3.client('s3')
bucket = sagemaker.Session().default_bucket()
prefix = "financial-usecase-mme"

# account mapping for SageMaker MME Triton Image
account_id_map = {
    "us-east-1": "785573368785",
    "us-east-2": "007439368137",
    "us-west-1": "710691900526",
    "us-west-2": "301217895009",
    "eu-west-1": "802834080501",
    "eu-west-2": "205493899709",
    "eu-west-3": "254080097072",
    "eu-north-1": "601324751636",
    "eu-south-1": "966458181534",
    "eu-central-1": "746233611703",
    "ap-east-1": "110948597952",
    "ap-south-1": "763008648453",
    "ap-northeast-1": "941853720454",
    "ap-northeast-2": "151534178276",
    "ap-southeast-1": "324986816169",
    "ap-southeast-2": "355873309152",
    "cn-northwest-1": "474822919863",
    "cn-north-1": "472730292857",
    "sa-east-1": "756306329178",
    "ca-central-1": "464438896020",
    "me-south-1": "836785723513",
    "af-south-1": "774647643957",
}

region = boto3.Session().region_name
if region not in account_id_map.keys():
    raise ("UNSUPPORTED REGION")

base = "amazonaws.com.cn" if region.startswith("cn-") else "amazonaws.com"
mme_triton_image_uri = (
    "{account_id}.dkr.ecr.{region}.{base}/sagemaker-tritonserver:22.12-py3".format(
        account_id=account_id_map[region], region=region, base=base
    )
)

## Workflow Overview

This section presents overview of main steps for preparing DistilBERT TensorFlow model (served using TensorFlow backend) 

### 1. Generate Model Artifacts

#### DistilBERT TensorFlow model

First, we use HuggingFace transformers to load pre-trained DistilBERT TensorFlow model that has been fine-tuned for sentiment analysis binary classification task. Then, we save the model as SavedModel serialized format. It can take a few minutes.

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
from transformers.utils import logging
# set logging to ERROR and above
logging.set_verbosity(40)
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

print("Downloading pretrained DistilBERT Classification TF Model from HuggingFace...")
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

save_model_dir = './workspace/hf_distilbert'
print("Serializing Model to SavedModel file...")
model.save_pretrained(save_model_dir, saved_model=True)

In [ ]:
save_model_dir

### 2. Build Model Respository

Using Triton on SageMaker requires us to first set up a [model repository](https://github.com/triton-inference-server/server/blob/main/docs/user_guide/model_repository.md) folder containing the models we want to serve. For each model we need to create a model directory consisting of the model artifact and define config.pbtxt file to specify [model configuration](https://github.com/triton-inference-server/server/blob/main/docs/model_configuration.md) which Triton uses to load and serve the model. 



#### DistilBERT TensorFlow Model

Model repository structure for DistilBERT TensorFlow Model.

```
distilbert_tf
├── 1
│   └── model.savedmodel
└── config.pbtxt
```

Model configuration must specify the platform and backend properties, max_batch_size property and the input and output tensors of the model. Additionally, you can specify instance_group and dynamic_batching properties for optimal inference performance in terms of latency and concurrency.

Below we set up the DistilBERT TensorFlow Model in the model repository:

In [ ]:
!mkdir -p model_repository/distilbert_tf/1
!cp -r workspace/hf_distilbert/saved_model/1 workspace/model.savedmodel
!cp -r workspace/model.savedmodel model_repository/distilbert_tf/1/

Then we define its config file:

In [ ]:
%%writefile model_repository/distilbert_tf/config.pbtxt
name: "distilbert_tf"
platform: "tensorflow_savedmodel"
max_batch_size: 8
input: [
    {
        name: "input_ids"
        data_type: TYPE_INT32
        dims: [ -1 ]
    },
    {
        name: "attention_mask"
        data_type: TYPE_INT32
        dims: [ -1 ]
    }
]
output: [
    {
        name: "logits"
        data_type: TYPE_FP32
        dims: [ 2 ]
    }
]
instance_group {
  count: 1
  kind: KIND_GPU
}
dynamic_batching {
}

### 3. Package models and upload to S3

Next, we will package our models as `*.tar.gz` files for uploading to S3. 

In [ ]:
!tar -C model_repository/ -czf distilbert_tf.tar.gz distilbert_tf
model_uri_distilbert_tf = sagemaker_session.upload_data(path="distilbert_tf.tar.gz", key_prefix=prefix)

In [ ]:
model_uri_distilbert_tf

### 4. Create SageMaker Endpoint

Now that we have uploaded the model artifacts to S3, we can create a SageMaker multi-model endpoint.

#### Define the serving container
In the container definition, define the `ModelDataUrl` to specify the S3 directory that contains all the models that SageMaker multi-model endpoint will use to load and serve predictions. Set `Mode` to `MultiModel` to indicate SageMaker would create the endpoint with MME container specifications. We set the container with an image that supports deploying multi-model endpoints with GPU

In [ ]:
model_data_url = f"s3://{bucket}/{prefix}/"

container = {
    "Image": mme_triton_image_uri,
    "ModelDataUrl": model_data_url,
    "Mode": "MultiModel",
}

#### Create a multi-model object

Once the image, data location are set we create the model using `create_model` by specifying the `ModelName` and the Container definition

In [ ]:
ts = time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())
sm_model_name = f"{prefix}-mdl-{ts}"

create_model_response = sm_client.create_model(
    ModelName=sm_model_name, ExecutionRoleArn=role, PrimaryContainer=container
)

print("Model Arn: " + create_model_response["ModelArn"])

#### Define configuration for the multi-model endpoint

Using the model above, we create an [endpoint configuration](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_CreateEndpointConfig.html) where we can specify the type and number of instances we want in the endpoint. Here we are deploying to `g5.2xlarge` NVIDIA GPU instance.

In [ ]:
endpoint_config_name = f"{prefix}-epc-{ts}"

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "InstanceType": "ml.g5.2xlarge",
            "InitialVariantWeight": 1,
            "InitialInstanceCount": 1,
            "ModelName": sm_model_name,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

#### Create Multi-Model Endpoint

Using the above endpoint configuration we create a new sagemaker endpoint and wait for the deployment to finish. The status will change to **InService** once the deployment is successful.

In [ ]:
endpoint_name = f"{prefix}-ep-{ts}"

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)

print("Endpoint Arn: " + create_endpoint_response["EndpointArn"])

## NOTE: we will use this endpoint_name in next notebook. 

In [ ]:
print("Endpoint Name: " + endpoint_name)

In [ ]:
resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

In [ ]:
%store sm_model_name
%store endpoint_config_name
%store endpoint_name

### 5. Run Inference

Once we have the endpoint running we can use some sample raw data to do an inference using JSON as the payload format. For the inference request format, Triton uses the KFServing community standard [inference protocols](https://github.com/triton-inference-server/server/blob/main/docs/protocol/README.md).

#### Add utility methods for preparing JSON request payload



We'll use the following utility methods to convert our inference request for DistilBERT and T5 models into a json payload.

In [ ]:
from transformers import AutoTokenizer

def get_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return tokenizer

def tokenize_text(model_name, text):
    tokenizer = get_tokenizer(model_name)
    tokenized_text = tokenizer(text, padding=True, return_tensors="np")
    return tokenized_text.input_ids, tokenized_text.attention_mask

def get_text_payload(model_name, text):
    input_ids, attention_mask = tokenize_text(model_name, text)
    payload = {}
    payload["inputs"] = []
    payload["inputs"].append({"name": "input_ids", "shape": input_ids.shape, "datatype": "INT32", "data": input_ids.tolist()})
    payload["inputs"].append({"name": "attention_mask", "shape": attention_mask.shape, "datatype": "INT32", "data": attention_mask.tolist()})
    return payload

#### Invoke target model on Multi Model Endpoint

We can send inference request to multi-model endpoint using `invoke_enpoint` API. We specify the `TargetModel` in the invocation call and pass in the payload for each model type.

#### DistilBERT TensorFlow Model

##### Sample DistilBERT Inference using Json Payload

First, we show some sample inference on the DistilBERT TensorFlow Binary Classification Model deployed on Triton's TensorFlow SavedModel Backend behind SageMaker MME GPU endpoint

In [ ]:
texts_to_classify = ["One of the challenges in the oil production in the North Sea is scale formation that can plug pipelines and halt production .",
                     "With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability ."]
batch_size = len(texts_to_classify)

distilbert_model = "distilbert-base-uncased-finetuned-sst-2-english"
distilbert_payload = get_text_payload(distilbert_model, texts_to_classify)

In [ ]:
response = runtime_sm_client.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType="application/octet-stream",
    Body=json.dumps(distilbert_payload),
    TargetModel="distilbert_tf.tar.gz",
)

response_body = json.loads(response["Body"].read().decode("utf8"))
logits = np.array(response_body["outputs"][0]["data"]).reshape(batch_size, -1)
CLASSES = ["NEGATIVE", "POSITIVE"]
predictions = []

for i in range(batch_size):
    pred_class_idx = np.argmax(logits[i])
    predictions.append(CLASSES[pred_class_idx])
print(predictions)

##### Sample DistilBERT Inference using Binary + Json Payload

We can also use `binary+json` as the payload format to get better performance for the inference call. The specification of this format is provided [here](https://github.com/triton-inference-server/server/blob/main/docs/protocol/extension_binary_data.md).

**Note:** With the binary+json format, we have to specify the length of the request metadata in the header to allow Triton to correctly parse the binary payload. This is done using a custom Content-Type header `Application/vnd.sagemaker-triton.binary+json;json-header-size={}`.

Please note, this is different from using `Inference-Header-Content-Length` header on a stand-alone Triton server since custom headers are not allowed in SageMaker.

The `tritonclient` package in Triton provides utility methods to generate the payload without having to know the details of the specification. We'll use the following method to convert our inference request for DistilBERT and T5 models into a binary format which provides lower latencies for inference.

In [ ]:
import tritonclient.http as httpclient
import numpy as np

def get_text_payload_binary(model_name, text):
    inputs = []
    outputs = []
    input_ids, attention_mask = tokenize_text(model_name, text)
    inputs.append(httpclient.InferInput("input_ids", input_ids.shape, "INT32"))
    inputs.append(httpclient.InferInput("attention_mask", attention_mask.shape, "INT32"))

    inputs[0].set_data_from_numpy(input_ids.astype(np.int32), binary_data=True)
    inputs[1].set_data_from_numpy(attention_mask.astype(np.int32), binary_data=True)
    
    output_name = "output" if model_name == "t5-small" else "logits"
    request_body, header_length = httpclient.InferenceServerClient.generate_request_body(
        inputs, outputs=outputs
    )
    return request_body, header_length

In [ ]:
request_body, header_length = get_text_payload_binary(distilbert_model, texts_to_classify)

response = runtime_sm_client.invoke_endpoint(EndpointName=endpoint_name,
                                  ContentType='application/vnd.sagemaker-triton.binary+json;json-header-size={}'.format(header_length),
                                  Body=request_body,
                                  TargetModel='distilbert_tf.tar.gz')

# Parse json header size length from the response
header_length_prefix = "application/vnd.sagemaker-triton.binary+json;json-header-size="
header_length_str = response['ContentType'][len(header_length_prefix):]
output_name = "logits"
# Read response body
result = httpclient.InferenceServerClient.parse_response_body(
    response['Body'].read(), header_length=int(header_length_str))
logits = result.as_numpy(output_name)
CLASSES = ["NEGATIVE", "POSITIVE"]
predictions = []

for i in range(batch_size):
    pred_class_idx = np.argmax(logits[i])
    predictions.append(CLASSES[pred_class_idx])
print(predictions)